In [3]:
import torch
import scipy as sp
import scipy.misc
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import imageio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms 
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
from torch.autograd import Variable
from PIL import Image
from PIL import ImageChops
from PIL import ImageEnhance
import torch.optim as optim
import tqdm
import copy
import utils
import nyu_dataset
import pix2pix_model

In [9]:
num_classes = int(utils.calculate_number_classes())

In [10]:
segmentation_dataset_train = nyu_dataset.SegmentationDataset(transforms=nyu_dataset.SegmentationTransform())
data_loader_train = DataLoader(segmentation_dataset_train, batch_size=8, shuffle=True, num_workers=1)

segmentation_dataset_validation = nyu_dataset.SegmentationDataset(transforms=
                                                                  nyu_dataset.SegmentationTransform(False), 
                                                                  val=True)
data_loader_val = DataLoader(segmentation_dataset_validation, batch_size=1, shuffle=False, num_workers=1)

In [11]:
generator = pix2pix_model.Generator(num_classes, 3)
discriminator = pix2pix_model.Discriminator(num_classes + 3)

In [ ]:
def train(data_loader_train, data_loader_val, generator, discriminator, num_epoch=100):
    d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    g_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    
    for epoch in range(num_epoch):
        for image, segmentation in data_loader_train:
            #TODO: training procedure.
                   